In [1]:
from tensorflow import keras
import tensorflow as tf
from keras import Input, Model, Sequential
from keras.layers import Flatten, Dense, Embedding, Concatenate, Conv1D, Lambda, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Concatenate, Flatten, Dot, Dropout
import keras.backend as K
from tensorflow.keras.metrics import AUC
import numpy as np
import glob
from tensorflow.keras.models import load_model

In [2]:
def lstm_model():    
    inputs = keras.Input(shape=(20, 8, 8, 34))

    # Convolutional patch embedding
    x = Conv2D(filters=16, kernel_size=(5, 5), activation='relu', padding='same')(inputs)
    x = Conv2D(filters=16, kernel_size=(4, 4), activation='relu', padding='same')(x)
    x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
    x = Conv2D(filters=64, kernel_size=(2, 2), activation='relu')(x)
    x = Conv2D(filters=64, kernel_size=(2, 2), activation='relu')(x)
    x = keras.layers.Reshape((20, 1024))(x)
    x = keras.layers.LSTM(32)(x)
    x = Flatten()(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss=keras.losses.BinaryCrossentropy(),
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        metrics=['accuracy', AUC(curve='ROC')],
    )
    return model

In [3]:
model = lstm_model()

2023-06-23 13:44:06.264361: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-23 13:44:17.880113: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46503 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:23:00.0, compute capability: 8.9


In [3]:
def replace_last(string, word_to_replace, replacement):
    split_string = string.rsplit(word_to_replace, 1)
    return replacement.join(split_string)

In [4]:
import gc
#Function to clean keras cache to allow calling the 'fit' function multiple times
def reset_keras(model):
    sess = K.get_session()
    K.clear_session()
    sess.close()
    sess = K.get_session()

    try:
        del model
    except:
        pass

    gc.collect()

In [5]:
files = glob.glob('/sise/liorrk-group/DataSets/Datamining/xs_data/xs*')
#First data file used for test set
Xs_test = np.load(files[0], allow_pickle=True) #Use first file for testing
ys_test = np.load(replace_last(files[0], 'xs', 'ys'), allow_pickle=True)

print(Xs_test.shape)
print(ys_test.shape)

def model_train_epoch():
    global model
    #Train set too big to load all at once, load each file separately and train, skip first file as it is for testing
    for file in files[1:]:
        Xs_train = np.load(file, allow_pickle=True)
        ys_train = np.load(replace_last(file, 'xs', 'ys'), allow_pickle=True)
        print(Xs_train.shape)
        print(ys_train.shape)
        #Reload model before each 'fit' call due to keras memory leak
        model.save("my_model_rnn")
        reset_keras(model)    
        model = load_model("my_model_rnn")
        
        model.fit(Xs_train, ys_train, epochs=1, batch_size=128)
        del Xs_train
        del ys_train
        gc.collect()
    score = model.evaluate(Xs_test, ys_test)
    print(f'Test set evaluation: {score}')

(195572, 20, 8, 8, 34)
(195572,)


In [8]:
epoch_count = 10
for i in range(epoch_count):
    model_train_epoch()

(195400, 20, 8, 8, 34)
(195400,)


2023-06-23 13:55:41.234730: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: my_model_rnn/assets


INFO:tensorflow:Assets written to: my_model_rnn/assets
2023-06-23 13:55:42.773513: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46503 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:23:00.0, compute capability: 8.9
2023-06-23 13:55:42.780218: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46503 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:23:00.0, compute capability: 8.9
2023-06-23 13:55:53.850799: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2023-06-23 13:56:20.338979: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8401
2023-06-23 13:56:47.245015: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This

1527/1527 [==============================] - 70s 8ms/step - loss: 0.4038 - accuracy: 0.8617 - auc: 0.5111
(196042, 20, 8, 8, 34)
(196042,)


...

INFO:tensorflow:Assets written to: my_model_rnn/assets
2023-06-24 14:02:40.124391: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46503 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:23:00.0, compute capability: 8.9


955/955 [==============================] - 8s 7ms/step - loss: 0.4217 - accuracy: 0.8406 - auc: 0.6405
(194148, 20, 8, 8, 34)
(194148,)


INFO:tensorflow:Assets written to: my_model_rnn/assets


INFO:tensorflow:Assets written to: my_model_rnn/assets
2023-06-24 14:09:11.209394: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46503 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:23:00.0, compute capability: 8.9


6112/6112 [==============================] - 17s 3ms/step - loss: 0.3684 - accuracy: 0.8830 - auc: 0.6020
Test set evaluation: [0.3684428334236145, 0.882958710193634, 0.6019802689552307]


In [6]:
model = load_model("my_model_rnn")

2023-06-25 18:02:22.196454: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-25 18:02:26.522208: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46503 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:61:00.0, compute capability: 8.9


In [8]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [10]:
y_pred = model.predict(Xs_test)

auc = roc_auc_score(ys_test, y_pred)
print("AUC:", auc)

auc_pr = average_precision_score(ys_test, y_pred)
print("AUC-PR:", auc_pr)

AUC: 0.5975479028907835
AUC-PR: 0.16201839671285723
